### Import dataset from robflow

# Model Detection Performance

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="m3zcgsb8zN6fHOST8Swy")
project = rf.workspace("doxariaproject").project("eya_-2bh")
version = project.version(9)
dataset = version.download("yolov8")


## Install HaggingFace And Libraries

In [ ]:
!git clone https://huggingface.co/toumix/yolo-medicament-dosage

In [ ]:
!pip install ultralytics paddleocr opencv-python-headless matplotlib --quiet

In [ ]:
!pip install paddlepaddle
!pip install paddleocr

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Runs File

In [ ]:
import os

path = "/content/drive/MyDrive/pi_project_3/datasets/runs"
if os.path.exists(path):
    rois_files = os.listdir(path)
    print(rois_files)
else:
    print("❌ Chemin introuvable : ", path)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Charger les métriques YOLOv8 depuis results.csv
csv_path = "/content/drive/MyDrive/pi_project_3/datasets/runs/results.csv"
results = pd.read_csv(csv_path)

# Vérifier les colonnes exactes
print(results.columns)

# Style seaborn
sns.set(style="whitegrid")

# Tracer Précision, Rappel et mAP50
plt.figure(figsize=(12, 6))
plt.plot(results['epoch'], results['metrics/precision(B)'], label='Precision')
plt.plot(results['epoch'], results['metrics/recall(B)'], label='Recall')
plt.plot(results['epoch'], results['metrics/mAP50(B)'], label='mAP50')  # <= correct !

plt.title('📊 Évolution des performances - YOLOv8L')
plt.xlabel('Époque')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(results['epoch'], results['train/box_loss'], label='Train Box Loss')
plt.plot(results['epoch'], results['train/cls_loss'], label='Train Class Loss')
plt.plot(results['epoch'], results['train/dfl_loss'], label='Train DFL Loss')
plt.plot(results['epoch'], results['val/box_loss'], label='Val Box Loss', linestyle='--')
plt.plot(results['epoch'], results['val/cls_loss'], label='Val Class Loss', linestyle='--')
plt.plot(results['epoch'], results['val/dfl_loss'], label='Val DFL Loss', linestyle='--')

plt.title('📉 Évolution des Losses (Train vs Validation)')
plt.xlabel('Époque')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


3️⃣ Courbe des métriques d'évaluation

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(results['epoch'], results['metrics/precision(B)'], label='Precision')
plt.plot(results['epoch'], results['metrics/recall(B)'], label='Recall')
plt.plot(results['epoch'], results['metrics/mAP50(B)'], label='mAP50')
plt.plot(results['epoch'], results['metrics/mAP50-95(B)'], label='mAP50-95')

plt.title('📊 Évolution des Metrics - YOLOv8L')
plt.xlabel('Époque')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.show()


4️⃣ Heatmap de Corrélation des Colonnes

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(results.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('🔥 Corrélation entre les variables d\'apprentissage')
plt.show()


5️⃣ Comparatif BoxPlot des Losses

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=results[['train/box_loss', 'train/cls_loss', 'train/dfl_loss',
                          'val/box_loss', 'val/cls_loss', 'val/dfl_loss']])
plt.xticks(rotation=45)
plt.title('📦 Distribution des Losses')
plt.show()


💡 Bonus : Tableau résumé

In [ ]:
print("Résumé des performances :\n")
print(results[['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']].describe())


💡 Étape 1 — Détection YOLO + Annotation des Images

In [ ]:
pip install easyocr


In [ ]:
import pandas as pd

df = pd.read_csv("/content/resultats_annotes/resultats_ocr.csv")
aucun_texte_count = df[df['texte_ocr'] == "Aucun texte"].shape[0]
total_count = df.shape[0]
proportion = (aucun_texte_count / total_count) * 100

print(f"Nombre total de crops traités : {total_count}")
print(f"Nombre de crops sans texte prédit ('Aucun texte') : {aucun_texte_count}")
print(f"Proportion : {proportion:.2f}%")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt  # <-- Ajout de l'import

# Charger le fichier CSV
df = pd.read_csv("/content/resultats_annotes/resultats_ocr.csv")

# Compter le nombre de prédictions "Aucun texte"
aucun_texte_count = df[df['texte_ocr'] == "Aucun texte"].shape[0]

# Compter le total de prédictions
total_count = df.shape[0]

# Créer les données pour le graphique
labels = ['Aucun texte', 'Texte présent']
sizes = [aucun_texte_count, total_count - aucun_texte_count]
colors = ['#ff9999', '#66b3ff']
explode = (0.1, 0)  # Mettre en avant "Aucun texte"

# Créer le graphique à secteurs
plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')  # Pour que le pie chart soit un cercle
plt.title("Proportion de textes prédits")
plt.show()


✅ 1️⃣ Lister les images qui posent souvent problème (beaucoup de "Aucun texte")

✅ 2️⃣ Logger les crops où l’OCR a échoué même après rotation
Dans ton code d’origine, tu peux déjà ajouter un log pendant la boucle :

Modifie la partie OCR comme ça :